<a href="https://colab.research.google.com/github/BOOKOFMEME1/SecLists/blob/master/bensipriser_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# URL till bensinpriser för Finspång
url = "https://bensinpriser.nu/stationer/9/ostergotlands-lan/finspang"

# Headers för begäran
headers = {
    "Cookie": "PHPSESSID=n6848mnmistcgsvkimafp15chl; FCCDCF=%5Bnull%2Cnull%2C%22null%22%5D",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "application/json",
    "Content-Type": "application/json",
}
money = 1000
# Skicka GET-förfrågan
response = requests.get(url, headers=headers)

# Kontrollera statuskod och skriv ut svaret
if response.status_code == 200:
    print("Lyckades hämta data!")
    soup = BeautifulSoup(response.text, 'lxml')

    # Hitta tabellen med bensinpriser
    table = soup.find('table', {'id': 'price_table'})
    if table:
        rows = table.find_all('tr')
        data = []
        for row in rows[1:]:  # Hoppa över rubrikraden
            columns = row.find_all('td')
            if len(columns) > 1:
                station = columns[0].get_text(strip=True)
                price = columns[1].get_text(strip=True)
                # Filtrera bort oönskade rader
                if "TIPS!" not in station and "Okänt" not in price:
                    # Extrahera endast det numeriska värdet från priset
                    price_value = re.search(r'\d+,\d+', price).group().replace(',', '.')
                    data.append({"Station": station, "Pris": price_value})

        # Skapa DataFrame
        df = pd.DataFrame(data)

        # Konvertera pris till numeriskt värde för sortering
        df['Pris'] = df['Pris'].astype(float)

        # Sortera DataFrame efter pris
        df = df.sort_values(by='Pris')

        # Skriv ut DataFrame
        print(df)

        # Skriv ut den billigaste stationen
        cheapest_station = df.iloc[0]
        print(f"Billigaste station: {cheapest_station['Station']}, Pris: {cheapest_station['Pris']} kr")

        # Beräkna besparingar
        df['Besparing'] = df['Pris'] - cheapest_station['Pris']

        # Skriv ut besparingar
        for index, row in df.iterrows():
            if row['Besparing'] > 0.10:  # Justera tröskelvärdet här
                savings_100kr = (money / row['Pris']) * row['Besparing']
                print(f"Du sparar {savings_100kr:.2f} kr om du tankar för {money} kr på {cheapest_station['Station']} istället för {row['Station']}")

        # Formatera DataFrame med linjer som skiljer
        df_styled = df.style.set_table_styles(
            [{'selector': 'tr:hover',
              'props': [('background-color', '#f5f5f5')]},
             {'selector': 'th',
              'props': [('border', '1px solid black')]},
             {'selector': 'td',
              'props': [('border', '1px solid black')]}]
        )
        print(df_styled)
    else:
        print("Kunde inte hitta prislistan.")
else:
    print(f"Fel: {response.status_code}")
    print(response.text)

Lyckades hämta data!
                                             Station   Pris
0                      St1FinspångÖstermalmsvägen 25  16.64
1                     PreemFinspångBergslagsvägen 77  16.74
2                   TankaFinspångNorrköpingsvägen 28  16.74
3                    OKQ8FinspångNorrköpingsvägen 26  16.89
4  Circle KFinspångLeverstorpsvägen 2, 612 44 Fin...  16.89
Billigaste station: St1FinspångÖstermalmsvägen 25, Pris: 16.64 kr
Du sparar 14.80 kr om du tankar för 1000 kr på St1FinspångÖstermalmsvägen 25 istället för OKQ8FinspångNorrköpingsvägen 26
Du sparar 14.80 kr om du tankar för 1000 kr på St1FinspångÖstermalmsvägen 25 istället för Circle KFinspångLeverstorpsvägen 2, 612 44 Finspång
